In [0]:
%run /Workspace/Users/surajss.3110@gmail.com/Learning/PySpark/MasterNoteBooks/INC_MasterNoteBook



In [0]:
inc = load_INC()
display(inc)

In [0]:
# 1. Ticket Volume Trend Detection Identify sudden spikes or drops in ticket volume over time using statistical thresholds or moving averages.

df1a = (
    inc
    .withColumn("Opened", expr("try_to_timestamp(Opened,'MM/dd/yyyy HH:mm:ss')"))
    .withColumn("Resolved", expr("try_to_timestamp(Resolved,'MM/dd/yyyy HH:mm:ss')"))
    .na.drop(subset=["Opened", "Resolved"])
    .withColumn("MonthName", month("Opened"))
    .groupby("MonthName")
    .count()
    .withColumnRenamed("count", "Count Of Incident in Month")
    .orderBy(col("MonthName"))
)

moving_avg_window = Window.orderBy("MonthName").rangeBetween(-2, 0)

df1a_with_ma = (
    df1a.withColumn("MovingAvg_3Month", avg("Count Of Incident in Month").over(moving_avg_window))
    .withColumn("Deviation",round(col("Count Of Incident in Month")-col("MovingAvg_3Month"),2))
    .withColumn("TrendFlag", when(col("Deviation") > 1, "Spike")
    .when(col("Deviation") < -1, "Drop")
    .otherwise("Normal")

    ))

display(df1a_with_ma)





Databricks visualization. Run in Databricks to view.

In [0]:
# Done by me 

#Resolution Time Distribution Analysis Create a histogram-like summary of resolution times bucketed into intervals (e.g., 0–2 hrs, 2–6 hrs, etc.).

# Resolution Time Distribution Analysis Create a histogram-like summary of resolution times bucketed into intervals (e.g., 0–2 hrs, 2–6 hrs, etc.).

df1b = (inc
        .withColumn("Opened", expr("try_to_timestamp(Opened,'MM/dd/yyyy HH:mm:ss')"))
        .withColumn("Resolved", expr("try_to_timestamp(Resolved,'MM/dd/yyyy HH:mm:ss')"))
        .withColumn("Resolved Timing in Hours", timestamp_diff("hour", "Opened", "Resolved"))
        .na.drop(subset=["Opened", "Resolved"])
        .withColumn("TrendFlag", 
                    when(col("Resolved Timing in Hours") <= 2, "Quick fixes")
                    .when((col("Resolved Timing in Hours") > 2) & (col("Resolved Timing in Hours") <= 6), "Moderate")
                    .when((col("Resolved Timing in Hours") > 6) & (col("Resolved Timing in Hours") <= 12), "Longer")
                    .when((col("Resolved Timing in Hours") > 12) & (col("Resolved Timing in Hours") <= 24), "Full-day issues")
                    .otherwise("Extended resolution"))
        .select("Issue", "Summary", "Opened", "Resolved", "Resolved Timing in Hours", "TrendFlag")
        )



final = (
    df1b.groupBy("TrendFlag")
    .count()
    .orderBy(col("count"))
)         


display(final)      

In [0]:
# Resolution Time Distribution Analysis: Create a histogram-like summary of resolution times bucketed into intervals (e.g., 0–2 hrs, 2–6 hrs, etc.).

df1b = (
    inc
    .withColumn("Opened", expr("try_to_timestamp(Opened,'MM/dd/yyyy HH:mm:ss')"))
    .withColumn("Resolved", expr("try_to_timestamp(Resolved,'MM/dd/yyyy HH:mm:ss')"))
    .na.drop(subset=["Opened", "Resolved"])
    .withColumn("Resolved_Timing_in_Hours", (unix_timestamp("Resolved") - unix_timestamp("Opened")) / 3600)
    .withColumn(
        "TrendFlag",
        when(col("Resolved_Timing_in_Hours") <= 2, "Quick fixes")
        .when((col("Resolved_Timing_in_Hours") > 2) & (col("Resolved_Timing_in_Hours") <= 6), "Moderate")
        .when((col("Resolved_Timing_in_Hours") > 6) & (col("Resolved_Timing_in_Hours") <= 12), "Longer")
        .when((col("Resolved_Timing_in_Hours") > 12) & (col("Resolved_Timing_in_Hours") <= 24), "Full-day issues")
        .otherwise("Extended resolution")
    )
    .select("Issue", "Summary", "Opened", "Resolved", "Resolved_Timing_in_Hours", "TrendFlag")
)

final = (
    df1b.groupBy("TrendFlag")
    .count()
    .orderBy(col("count"))
)

display(final)

Databricks visualization. Run in Databricks to view.